In [1]:
from util.fcn8s import fcn8s
from util.generator import Generator
from util.layer import Conv_Block
from tensorflow.keras import optimizers
import tensorflow as tf


In [2]:
fcn = fcn8s(10)
model=fcn.create_model()

adam=optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=adam, loss='categorical_crossentropy',
              metrics=['accuracy'])

with open('./train.txt') as f:
    image_path_list = f.read().split('\n')
    image_path_list.remove('')

with open('./class.txt') as f:
    class_path_list = f.read().split('\n')
    class_path_list.remove('')

In [3]:
from tensorflow.keras.callbacks import ModelCheckpoint
import os

Model_save='./model'

checkpoint = ModelCheckpoint(
    filepath=os.path.join(Model_save,"FCN_model-{epoch:02d}.h5"),
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,
    mode='min'
)

In [4]:
val_count = int(len(image_path_list)*0.1)

train_gen = Generator(
    image_path_list[val_count:], class_path_list[val_count:], 256, 256, 10, 2)

val_gen = Generator(
    image_path_list[:val_count], class_path_list[:val_count], 256, 256, 10, 2)

In [5]:
model.fit(train_gen, steps_per_epoch=train_gen.batches_per_epoch, validation_data=val_gen,
                    validation_steps=val_gen.batches_per_epoch, epochs=1, shuffle=True,callbacks=[checkpoint])

553/553 [==============================] - 370s 670ms/step - loss: 1.8418 - accuracy: 0.3603 - val_loss: 1.5199 - val_accuracy: 0.4807


In [6]:
model.save('model/FCN_model-50.h5')